In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 4
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
SIM = False

In [2]:
# Import des paquets requis et configuration du ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
if SIM:
    environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-07:11311/" # X = votre numéro de Dingo
    environ['ROS_IP'] = "192.168.0.84" # adresse IP de votre station de travail
import rospy
from std_msgs.msg import Float32MultiArray
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose
from sensor_msgs.msg import Imu

In [3]:
# Création d'un nouveau contenant ROSBAG (en écriture)
import rosbag

if SIM:
    test_bag = rosbag.Bag('output_sim.bag', 'w')
else:
    test_bag = rosbag.Bag('output_real.bag', 'w')

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [4]:
# Compléter les variables suivantes avec les dimensions du robot en [m]
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045

In [5]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_controller', anonymous=True)

In [6]:
# Position subscriber callback
pose_msg = Pose()
def joint_callback(msg):
    global pose_msg
    pose_msg  = msg.pose.pose

In [7]:
# UMI subscriber callback
imu_msg = Imu()
def imu_callback(msg):
    global imu_msg
    imu_msg = msg

In [8]:
# ROS subscribers et publishers
if SIM:
    cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
    pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, joint_callback)
    imu_sub = rospy.Subscriber('/imu/data', Imu, imu_callback)
else:
    cmd_drive_pub = rospy.Publisher('/mobile_manip/base/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
    pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, joint_callback)
    imu_sub = rospy.Subscriber('/mobile_manip/base/imu/data', Imu, imu_callback)

In [9]:
# Impression des valeurs UMI et des encodeurs - pour tester
print("Imu data: ")
print("Orientation (quaternion): ", 
    imu_msg.orientation.x, imu_msg.orientation.y, 
    imu_msg.orientation.z, imu_msg.orientation.w
)
print("Angular velocity: ", 
    imu_msg.angular_velocity.x, imu_msg.angular_velocity.y, 
    imu_msg.angular_velocity.z
)
print("Linear acceleration: ", 
    imu_msg.linear_acceleration.x, imu_msg.linear_acceleration.y, 
    imu_msg.linear_acceleration.z
)
print("Position: ", 
    pose_msg.position.x, pose_msg.position.y
)

Imu data: 
Orientation (quaternion):  0.45781312018637765 0.4363733291533301 0.1676766442976508 0.7562208721966961
Angular velocity:  0.00047696783440187573 -0.0037464601919054985 -0.001875896006822586
Linear acceleration:  -0.08703505247831345 0.08976531028747559 0.031133651733398438
Position:  -0.04286426305770874 0.2029949575662613


# Explication calcul

#### On établi la relation entre la vitesse linéaire et angulaire du robot
vit_lin = vit_ang * (d/2)

vit_lin = vit_ang * 1/2

vit_ang = 2 * vit_lin


#### À partir des deux equations suivantes

vit_lin = (v_d + v_g) / 2

vit_ang = (v_d - v_g) / inter_wheel


#### On peut isoler les valeurs des vitesses de roues de droite et gauche

v_d = vit_lin + (inter_wheel * vit_ang) / 2

v_g = vit_lin - (inter_wheel * vit_ang) / 2

#### On substitue la vitesse angulaire par la relation trouvée precedemment pour un diamètre de 1

v_d = vit_lin + (inter_wheel * 2 * vit_lin) / 2

v_d = vit_lin + inter_wheel * vit_lin

v_d = vit_lin * (1 + inter_wheel)

v_d_ang = vit_lin * (1 + inter_wheel) / right_wheel_radius


v_g = vit_lin - (inter_wheel * 2 * vit_lin) / 2

v_g = vit_lin - inter_wheel * vit_lin

v_g = vit_lin * (1 - inter_wheel)

v_g_ang = vit_lin * (1 - inter_wheel) / left_wheel_radius

In [10]:
LINEAR = True
# Déterminez les valeurs numériques de vitesse aux roues requises pour touner sur un cercle R=1m

cmd_drive_msg = Drive()

# Commande de la vitesse lineaire
if LINEAR == True:
    v = 0.2
    cmd_drive_msg.drivers[1] = v*(1+interwheel_distance)/right_wheel_radius
    cmd_drive_msg.drivers[0] = v*(1-interwheel_distance)/left_wheel_radius
# Commande de la vitesse angulaire
else:
    w=0.3
    cmd_drive_msg.drivers[1] = (w/2)*(1+interwheel_distance)/right_wheel_radius
    cmd_drive_msg.drivers[0] = (w/2)*(1-interwheel_distance)/left_wheel_radius
    


print('Vitesse de droite: ', cmd_drive_msg.drivers[1])
print('Vitesse de gauche: ', cmd_drive_msg.drivers[0])

#Envoi des commandes au roues par topic ROS
cmd_drive_pub.publish(cmd_drive_msg)

Vitesse de droite:  6.488888888888889
Vitesse de gauche:  2.4000000000000004


In [11]:
# Enregistrer la trajectoire pendant 20 secondes
start = float(rospy.Time().now().secs)
rate = rospy.Rate(50)
while (float(rospy.Time().now().secs) - start) < 20:
    test_bag.write('/imu', imu_msg, rospy.Time().now())
    test_bag.write('/chassis_pose', pose_msg, rospy.Time().now())
    rate.sleep()

In [12]:
# arrêter le robot
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0

# Envoi des commandes au roues par topic ROS
cmd_drive_pub.publish(cmd_drive_msg)

In [13]:
# close the rosbag
test_bag.close()